# Humanitarian Aid Optimization: Identifying High-Need Regions Through Data Analysis

<div style="display: flex;">
  <div style="flex: 2;">
    <p>
      Представьте, что вы работаете в международной гуманитарной организации, главная задача которой — борьба с бедностью и помощь людям из наименее развитых стран. Для выполнения своей миссии ваша организация периодически финансирует различные проекты, поддерживающие нуждающиеся регионы.
    </p>
    <p>
      Вы собрали 100 миллионов долларов и должны принять решение, как распределить эти деньги — в какие страны их направить в первую очередь. Как выбрать, кто сильнее всего нуждается в финансовой поддержке?
    </p>
    <p>
      Конечно, можно взять страны с самыми низкими доходами населения или с самой маленькой продолжительностью жизни. А может, лучше помочь странам, у которых самая высокая инфляция? По отдельным признакам выбирать достаточно сложно — нам хотелось бы выявить группы стран, в которых в целом ситуация вызывает опасения по совокупности значений всех признаков.
    </p>
  </div>
  <div style="flex: 1;">
    <img src="assets/img1.jpg" alt="Описание изображения" style="width:50%;">
    <p style="text-align: center; font-size: small;">Источник изображения</p>
  </div>
</div>




Данные содержат следующие признаки:

country — название страны;
child_mort — показатель смертности детей до 5 лет;
exports — показатель экспорта на душу населения;
health — показатель затрат на здравоохранение на душу населения;
imports — показатель импорта на душу населения;
Income — средний доход на душу населения;
Inflation — годовой показатель инфляции;
life_expec — средняя продолжительность жизни;
total_fer — суммарный коэффициент рождаемости;
gdpp — значение ВВП на душу населения.

In [11]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.decomposition import PCA

In [4]:
# Загрузим данные из CSV файла

data = pd.read_csv('data/Country-data.csv')
data.head

<bound method NDFrame.head of                  country  child_mort  exports  health  imports  income  \
0            Afghanistan        90.2     10.0    7.58     44.9    1610   
1                Albania        16.6     28.0    6.55     48.6    9930   
2                Algeria        27.3     38.4    4.17     31.4   12900   
3                 Angola       119.0     62.3    2.85     42.9    5900   
4    Antigua and Barbuda        10.3     45.5    6.03     58.9   19100   
..                   ...         ...      ...     ...      ...     ...   
162              Vanuatu        29.2     46.6    5.25     52.7    2950   
163            Venezuela        17.1     28.5    4.91     17.6   16500   
164              Vietnam        23.3     72.0    6.84     80.2    4490   
165                Yemen        56.3     30.0    5.18     34.4    4480   
166               Zambia        83.1     37.0    5.89     30.9    3280   

     inflation  life_expec  total_fer   gdpp  
0         9.44        56.2       5

In [5]:
# Найдем максимальное значение показателя средней продолжительности жизни (life_expec)
max_life_expec = data['life_expec'].max()

# Округлим результат до одного знака после точки
max_life_expec_rounded = round(max_life_expec, 1)

max_life_expec_rounded

82.8

In [7]:
# Выделим признаки для стандартизации (исключая столбец с названиями стран)
features = data.columns.drop('country')

# Применим StandardScaler ко всем значениям признаков
scaler = StandardScaler()
data[features] = scaler.fit_transform(data[features])

# Получим значение первого признака для первого объекта
first_feature_first_object = data.iloc[0, 1]  # Первый объект и первый признак (т.к. 'country' - первый столбец)

# Округлим результат до двух знаков после точки
first_feature_first_object_rounded = round(first_feature_first_object, 2)

first_feature_first_object_rounded

1.29

In [10]:
# Построим матрицу корреляций для всех признаков
correlation_matrix = data[features].corr()

# Найдем максимальный коэффициент корреляции, исключая единицы на диагонали
max_correlation = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool)).max().max()

# Округлим результат до двух знаков после точки
max_correlation_rounded = round(max_correlation, 2)

max_correlation_rounded


0.9

In [12]:
# Применим PCA к стандартизированным данным
pca = PCA()
pca.fit(data[features])

# Найдем кумулятивную долю объясненной дисперсии
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)

# Определим минимальное количество компонент для объяснения 90% дисперсии
num_components = np.argmax(cumulative_variance >= 0.90) + 1

num_components

5

In [13]:
# Доля дисперсии, объясняемая первой главной компонентой
first_component_variance = pca.explained_variance_ratio_[0]

# Округлим результат до двух знаков после точки
first_component_variance_rounded = round(first_component_variance, 2)

first_component_variance_rounded


0.46

In [14]:
# Преобразуем данные в пространство главных компонент
pca_transformed_data = pca.transform(data[features])

# Создадим DataFrame из этих данных для удобства
pca_transformed_df = pd.DataFrame(pca_transformed_data)

# Построим матрицу корреляций для главных компонент
pca_correlation_matrix = pca_transformed_df.corr()

# Подсчитаем количество значений в матрице корреляций, которые превышают 0.01, исключая единичные значения на главной диагонали
high_correlation_count = np.sum((pca_correlation_matrix.abs() > 0.01) & (pca_correlation_matrix.abs() < 1))

high_correlation_count


c:\Users\mazur\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
dtype: int64

In [15]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Диапазон возможного количества кластеров
cluster_range = range(2, 11)
best_score = -1
best_n_clusters = 0

# Перебор количества кластеров и вычисление коэффициента силуэта
for n_clusters in cluster_range:
    kmeans = KMeans(n_clusters=n_clusters, init='k-means++', random_state=1)
    cluster_labels = kmeans.fit_predict(pca_transformed_df)
    silhouette_avg = silhouette_score(pca_transformed_df, cluster_labels)
    if silhouette_avg > best_score:
        best_score = silhouette_avg
        best_n_clusters = n_clusters

best_n_clusters


c:\Users\mazur\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Users\mazur\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Users\mazur\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Users\mazur\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value 

4

In [16]:
# Реализуем алгоритм k-means++ с оптимальным количеством кластеров
kmeans = KMeans(n_clusters=best_n_clusters, init='k-means++', random_state=1)
cluster_labels = kmeans.fit_predict(pca_transformed_df)

# Добавим метки кластеров к оригинальным данным
data['cluster'] = cluster_labels

# Найдем страну с самым высоким показателем детской смертности и самым низким ВВП на душу населения
max_child_mort_country = data.loc[data['child_mort'].idxmax()]
min_gdpp_country = data.loc[data['gdpp'].idxmin()]

# Определим кластеры для этих стран
max_child_mort_cluster = max_child_mort_country['cluster']
min_gdpp_cluster = min_gdpp_country['cluster']

max_child_mort_cluster, min_gdpp_cluster


c:\Users\mazur\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


(1, 1)

In [17]:
# Определяем кластер, в который попали страны с самым высоким показателем детской смертности и самым низким ВВП на душу населения
target_cluster = max_child_mort_cluster  # или min_gdpp_cluster, они должны быть одинаковыми

# Отбираем страны из целевого кластера
target_cluster_countries = data[data['cluster'] == target_cluster]

# Выбираем три страны с самой маленькой продолжительностью жизни
three_lowest_life_expec_countries = target_cluster_countries.nsmallest(3, 'life_expec')

# Получаем названия этих стран
three_lowest_life_expec_countries_names = three_lowest_life_expec_countries['country'].values

three_lowest_life_expec_countries_names


array(['Haiti', 'Lesotho', 'Central African Republic'], dtype=object)